# Imports and Mounting

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import os
import random
import shutil
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torch.optim as optim

from PIL import Image
from torch.utils.data import Dataset

!pip install tqdm
from tqdm import tqdm

!pip install rdkit
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

from collections import defaultdict
import re


# Sampling the data

In [248]:
project_root = "/content/drive/Shareddrives/CIS5190FinalProj"

handdrawn_root = "/content/drive/Shareddrives/CIS5190FinalProj/DECIMER_HDM_Dataset_Images"
computer_root = "/content/drive/Shareddrives/CIS5190FinalProj/Img2Mol"

In [249]:
print(f"There are {len(os.listdir(handdrawn_root))} handdrawn images")
print(f"There are {len(os.listdir(computer_root))} computer generated images")

There are 5088 handdrawn images
There are 10880 computer generated images


In [250]:
## Training : Validation : Testing = 7 : 2 : 1
NUMBER_OF_TRAINING_IMGS = 7000
NUMBER_OF_VAL_IMGS = NUMBER_OF_TRAINING_IMGS // 7 * 2
NUMBER_OF_TESTING_IMGS = NUMBER_OF_TRAINING_IMGS // 7

train_root = project_root + "/Train0.5"
val_root = project_root + "/Val0.5"
test_root = project_root + "/Test0.5"

## Creating image sets

In [251]:
def create_new_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

In [252]:
## Randomly sample the images
def split_data(comp_gen_percentage=0.5):
    if comp_gen_percentage < 0.5:
        print("Computer generated percentage cannot be smaller than 0.5")
        return

    # Computer generated images
    num_comp_gen_imgs_train = int(comp_gen_percentage * NUMBER_OF_TRAINING_IMGS)
    num_comp_gen_imgs_val = int(comp_gen_percentage * NUMBER_OF_VAL_IMGS)
    num_comp_gen_imgs_test = int(comp_gen_percentage * NUMBER_OF_TESTING_IMGS)


    files = os.listdir(computer_root) # a list of names of computer-generated images
    random.shuffle(files)

    comp_gen_filenames_train = files[:num_comp_gen_imgs_train]
    comp_gen_filenames_val = files[num_comp_gen_imgs_train: num_comp_gen_imgs_train+num_comp_gen_imgs_val]
    comp_gen_filenames_test = files[num_comp_gen_imgs_train+num_comp_gen_imgs_val:]

    # Create the directories
    create_new_directory(train_root)
    create_new_directory(val_root)
    create_new_directory(test_root)

    # Copy the images into the directories
    for filename in comp_gen_filenames_train:
        shutil.copy(os.path.join(computer_root, filename), train_root)
    for filename in comp_gen_filenames_val:
        shutil.copy(os.path.join(computer_root, filename), val_root)
    for filename in comp_gen_filenames_test:
        shutil.copy(os.path.join(computer_root, filename), test_root)



    # Handdrawn images
    num_hand_imgs_train = int((1 - comp_gen_percentage) * NUMBER_OF_TRAINING_IMGS)
    num_hand_imgs_val = int((1 - comp_gen_percentage) * NUMBER_OF_VAL_IMGS)
    num_hand_imgs_test = int((1 - comp_gen_percentage) * NUMBER_OF_TESTING_IMGS)


    files = os.listdir(handdrawn_root)
    random.shuffle(files)

    hand_filenames_train = files[:num_hand_imgs_train]
    hand_filenames_val = files[num_hand_imgs_train: num_hand_imgs_train+num_hand_imgs_val]
    hand_filenames_test = files[num_hand_imgs_train+num_hand_imgs_val:]

    # Copy the images into the directories
    for filename in hand_filenames_train:
        shutil.copy(os.path.join(handdrawn_root, filename), train_root)
    for filename in hand_filenames_val:
        shutil.copy(os.path.join(handdrawn_root, filename), val_root)
    for filename in hand_filenames_test:
        shutil.copy(os.path.join(handdrawn_root, filename), test_root)

In [ ]:
## Only need to run this once
#split_data(1)

## Loading the data into a dataloader

### Get the labels

In [253]:
periodic_table = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne",
                  "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca",
                  "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
                  "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr", "Y", "Zr",
                  "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn",
                  "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", "Nd",
                  "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb",
                  "Lu", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg",
                  "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th",
                  "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm",
                  "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds",
                  "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"]


In [254]:
def smiles_to_formula(smiles):
  mol = Chem.MolFromSmiles(smiles)
  formula = CalcMolFormula(mol)
  return formula

def formula_to_atoms(formula):
  atoms = defaultdict(int)
  elements = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
  for element, count in elements:
    count = int(count) if count else 1
    atoms[element] += count
  return dict(atoms)


def smiles_to_atoms(smiles):
  formula = smiles_to_formula(smiles)
  return formula_to_atoms(formula)

def atoms_to_array(atoms):
  # Initialize array with zeros for each element
  element_array = [0] * 118
  # Place the atom counts into the array based on the dictionary
  for element, count in atoms.items():
      if element in periodic_table:
          index = periodic_table.index(element)
          element_array[index] = count
  return element_array

def atoms_to_binary_array(atoms):
    element_array = [0] * 118
    for element in atoms:
        if element in periodic_table:
            index = periodic_table.index(element)
            element_array[index] = 1
    return element_array


In [255]:
handdrawn_df = pd.read_csv(project_root + "/DECIMER_HDM_Dataset_SMILES.tsv", sep='\t')
handdrawn_smiles_dict = handdrawn_df.set_index('IDs')['SMILES'].to_dict()

# Process each SMILES to convert into atom arrays
handdrawn_atom_arrays = {key: atoms_to_array(smiles_to_atoms(value)) for key, value in handdrawn_smiles_dict.items()}
handdrawn_atom_binary_arrays = {key: atoms_to_binary_array(smiles_to_atoms(value)) for key, value in handdrawn_smiles_dict.items()}

# To verify the transformation, print the first 5 elements of the transformed dictionary
for key in list(handdrawn_atom_arrays.keys())[:5]:
    print(key, ":", handdrawn_atom_arrays[key])
for key in list(handdrawn_atom_binary_arrays.keys())[:5]:
    print(key, ":", handdrawn_atom_binary_arrays[key])

CDK_Depict_1_2 : [3, 0, 0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CDK_Depict_1_4 : [6, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CDK_Depict_1_5 : [45, 0, 0, 0, 0, 21, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [256]:
with open("/content/drive/Shareddrives/CIS5190FinalProj/Img2Mol_map.pkl", 'rb') as f:
    comp_gen_df = pickle.load(f)
comp_gen_smiles_dict = comp_gen_df.set_index('Image')['SMILES'].to_dict()
comp_gen_atom_arrays = {key: atoms_to_array(smiles_to_atoms(value)) for key, value in comp_gen_smiles_dict.items()}
comp_gen_atom_binary_arrays = {key: atoms_to_binary_array(smiles_to_atoms(value)) for key, value in comp_gen_smiles_dict.items()}

In [257]:
# Verify transformation for both datasets by printing the first 5 elements
print("Hand-drawn dataset first 5 atom arrays:")
for key in list(handdrawn_atom_arrays.keys())[:5]:
    print(key, ":", handdrawn_atom_arrays[key])

print("\nComputer-generated dataset first 5 atom arrays:")
for key in list(comp_gen_atom_arrays.keys())[:5]:
    print(key, ":", comp_gen_atom_arrays[key])

print("Hand-drawn dataset first 5 atom arrays:")
for key in list(handdrawn_atom_binary_arrays.keys())[:5]:
    print(key, ":", handdrawn_atom_binary_arrays[key])

print("\nComputer-generated dataset first 5 atom arrays:")
for key in list(comp_gen_atom_binary_arrays.keys())[:5]:
    print(key, ":", comp_gen_atom_binary_arrays[key])

Hand-drawn dataset first 5 atom arrays:
CDK_Depict_1_2 : [3, 0, 0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CDK_Depict_1_4 : [6, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CDK_Depict_1_5 : [45, 0, 0, 0, 0, 21, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [258]:
## Combine the two dictionaries together
combined_mapping = handdrawn_atom_arrays.copy()
combined_binary_mapping = handdrawn_atom_binary_arrays.copy()

for key, value in comp_gen_atom_arrays.items():
    # Remove .png from the key
    key = key.replace('.png', '')
    combined_mapping[key] = value

for key, value in comp_gen_atom_binary_arrays.items():
    # Remove .png from the key
    key = key.replace('.png', '')
    combined_binary_mapping[key] = value

In [260]:
## Returns the label based on the filename of the image

def get_binary_label(filename):
    return combined_binary_mapping.get(filename, None)

In [261]:
def validate_dataset(image_names, root_dir, label_dict):
    valid_image_names = []
    for img_name in image_names:
        img_path = os.path.join(root_dir, img_name)
        if os.path.isfile(img_path) and img_name.replace(".png", "") in label_dict:
            valid_image_names.append(img_name)
    return valid_image_names


In [262]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, label_dict, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_names = validate_dataset(os.listdir(root_dir), root_dir, label_dict)
        self.label_dict = label_dict

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.label_dict[img_name.replace(".png", "")]

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.float32)
        return image, label


In [263]:
## Transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [264]:
train_binary_dataset = CustomDataset(root_dir=train_root, label_dict=combined_binary_mapping, transform=transform)
val_binary_dataset = CustomDataset(root_dir=val_root, label_dict=combined_binary_mapping, transform=transform)
test_binary_dataset = CustomDataset(root_dir=test_root, label_dict=combined_binary_mapping, transform=transform)

train_binary_loader = DataLoader(train_binary_dataset, batch_size=32, shuffle=True, num_workers=0)
val_binary_loader = DataLoader(val_binary_dataset, batch_size=32, shuffle=False, num_workers=0)
test_binary_loader = DataLoader(test_binary_dataset, batch_size=32, shuffle=False, num_workers=0)

Single point for sanity check

In [265]:
single_item_index = 0
single_item_binary_dataset = Subset(train_binary_dataset, [single_item_index])
single_item_binary_loader = DataLoader(single_item_binary_dataset, batch_size=1, shuffle=False)
for single_image, single_label in single_item_binary_loader:
    print("Image shape:", single_image.shape)
    print("Label:", single_label)



Image shape: torch.Size([1, 3, 256, 256])
Label: tensor([[1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


# Data Augmentation

In [266]:
aug1 = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # Random horizontal flip
    transforms.RandomRotation(degrees=15) # Random rotation by up to 15 degrees
])

aug2 = transforms.Compose([
    transforms.RandomResizedCrop(size=224),  # Random resized crop to 224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10)  # Random affine transformations
])

aug3 = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # Random horizontal flip
    transforms.RandomRotation(degrees=15), # Random rotation by up to 15 degrees
    transforms.RandomResizedCrop(size=224),  # Random resized crop to 224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10)  # Random affine transformations
])

# Model Pipeline (Transfer Learning)

In [267]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [357]:
def calculate_accuracy(preds, labels):
    # Ensure both predictions and labels are binary (0 or 1)
    preds = preds.round().int()
    labels = labels.int()

    # Count matches where both prediction and label are 1
    correct_ones = (preds & labels).sum().item()

    # Count total cases where either prediction or label is 1
    total_ones = ((preds == 1) | (labels == 1)).sum().item()

    # Calculate accuracy based on matches and total relevant cases
    accuracy = correct_ones / total_ones if total_ones > 0 else 0.0
    return accuracy


In [342]:
def calculate_batch_accuracy(outputs, labels):
    probs = torch.sigmoid(outputs)
    preds = (probs >= 0.5).float()

    # Convert tensors to integer type for bitwise operations
    preds_int = preds.int()
    labels_int = labels.int()

    # Correct matches where both prediction and label are 1
    correct_ones = (preds_int & labels_int).sum().item()

    # Total relevant cases where either prediction or label is 1
    total_ones = ((preds_int == 1) | (labels_int == 1)).sum().item()

    # Calculate accuracy based on matches and total relevant cases
    accuracy = correct_ones / total_ones if total_ones > 0 else 0.0
    return accuracy


In [358]:
def train_single_point(model, data_loader, criterion, optimizer, num_epochs=20):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        correct_train = 0.0
        total_train = 0

        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to the correct device

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            epoch_loss += loss.item()

            # Calculate batch accuracy
            batch_correct_train = calculate_batch_accuracy(outputs, labels)
            correct_train += batch_correct_train * labels.size(0)
            total_train += labels.size(0)

        epoch_loss /= len(data_loader)
        epoch_accuracy = correct_train / total_train

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')


In [310]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5, log_interval=10):
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0.0
        total_train = 0

        print(f"Epoch [{epoch+1}/{num_epochs}]")

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            batch_correct_train = calculate_batch_accuracy(outputs, labels)
            correct_train += batch_correct_train * labels.size(0)  # Accumulate correct predictions
            total_train += labels.size(0)  # Accumulate total predictions

            if batch_idx % log_interval == 0:
                current_train_loss = running_loss / (batch_idx + 1)
                current_train_accuracy = correct_train / total_train
                print(f"Train Batch [{batch_idx}/{len(train_loader)}], Loss: {current_train_loss:.4f}, Accuracy: {current_train_accuracy:.4f}")

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct_train / total_train

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct_val = 0.0
        total_val = 0

        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(val_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                batch_correct_val = calculate_batch_accuracy(outputs, labels)
                correct_val += batch_correct_val * labels.size(0)  # Accumulate correct predictions
                total_val += labels.size(0)  # Accumulate total predictions

                if batch_idx % log_interval == 0:
                    current_val_loss = val_loss / (batch_idx + 1)
                    current_val_accuracy = correct_val / total_val
                    print(f"Val Batch [{batch_idx}/{len(val_loader)}], Loss: {current_val_loss:.4f}, Accuracy: {current_val_accuracy:.4f}")

        val_loss = val_loss / len(val_loader)
        val_accuracy = correct_val / total_val

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        # Append values for this epoch to the lists
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

    return train_losses, train_accuracies, val_losses, val_accuracies


In [354]:
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    test_accuracy = 0.0
    total_batches = len(test_loader)

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(tqdm(test_loader, desc="Testing")):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Calculate accuracy using the provided function
            batch_accuracy = calculate_batch_accuracy(outputs, labels)
            test_accuracy += batch_accuracy

            # Debugging: Print predictions and targets
            if i % 50 == 0:
                # Apply sigmoid and threshold to show rounded predictions
                probs = torch.sigmoid(outputs)
                preds = (probs >= 0.5).float()
                print(f'Batch {i}/{total_batches}, Current Test Loss: {test_loss/(i+1):.4f}, Current Test Accuracy: {test_accuracy/(i+1):.4f}')
                print("Predictions:", preds[:5].cpu().numpy())
                print("Targets:", labels[:5].cpu().numpy())

    test_loss /= total_batches
    test_accuracy /= total_batches

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    return test_loss, test_accuracy

## ResNet

In [334]:
weights = ResNet18_Weights.DEFAULT

In [335]:
num_elements = 118

In [337]:
resnet_binary = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
for param in resnet_binary.parameters():
    param.requires_grad = True  # Unfreeze all layers

num_binary_features = resnet_binary.fc.in_features
resnet_binary.fc = nn.Linear(num_binary_features, num_elements)
resnet_binary = resnet_binary.to(device)  # Move model to GPU

In [362]:
# Load Single Point ResNet model
resnet_single_binary = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
for param in resnet_single_binary.parameters():
    param.requires_grad = True  # Unfreeze all layers

num_single_binary_features = resnet_single_binary.fc.in_features
resnet_single_binary.fc = nn.Linear(num_single_binary_features, num_elements)
resnet_single_binary = resnet_single_binary.to(device)  # Move model to GPU

In [363]:
criterion = nn.BCEWithLogitsLoss()
optimizer_single = torch.optim.SGD(resnet_single_binary.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.SGD(resnet_binary.parameters(), lr=0.001, momentum=0.9)

In [364]:
train_single_point(resnet_single_binary, single_item_binary_loader, criterion, optimizer_single, num_epochs=100)
test_model(resnet_single_binary, single_item_binary_loader, criterion)

Epoch [1/100], Loss: 0.7568, Accuracy: 0.0423
Epoch [2/100], Loss: 0.7555, Accuracy: 0.0423
Epoch [3/100], Loss: 0.7530, Accuracy: 0.0423
Epoch [4/100], Loss: 0.7495, Accuracy: 0.0423
Epoch [5/100], Loss: 0.7453, Accuracy: 0.0429
Epoch [6/100], Loss: 0.7403, Accuracy: 0.0435
Epoch [7/100], Loss: 0.7349, Accuracy: 0.0435
Epoch [8/100], Loss: 0.7289, Accuracy: 0.0448
Epoch [9/100], Loss: 0.7226, Accuracy: 0.0455
Epoch [10/100], Loss: 0.7159, Accuracy: 0.0462
Epoch [11/100], Loss: 0.7089, Accuracy: 0.0492
Epoch [12/100], Loss: 0.7017, Accuracy: 0.0492
Epoch [13/100], Loss: 0.6942, Accuracy: 0.0667
Epoch [14/100], Loss: 0.6865, Accuracy: 0.0862
Epoch [15/100], Loss: 0.6786, Accuracy: 0.0909
Epoch [16/100], Loss: 0.6705, Accuracy: 0.0962
Epoch [17/100], Loss: 0.6624, Accuracy: 0.1000
Epoch [18/100], Loss: 0.6542, Accuracy: 0.1020
Epoch [19/100], Loss: 0.6459, Accuracy: 0.1042
Epoch [20/100], Loss: 0.6377, Accuracy: 0.1064
Epoch [21/100], Loss: 0.6295, Accuracy: 0.1111
Epoch [22/100], Loss: 

Testing: 100%|██████████| 1/1 [00:00<00:00, 50.52it/s]

Batch 0/1, Current Test Loss: 0.2541, Current Test Accuracy: 1.0000
Predictions: [[1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Targets: [[1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Test Loss: 0.2541, Test Accuracy: 1.0000


(0.25409460067749023, 1.0)

In [2]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(resnet_binary.parameters(), lr=0.001, momentum=0.9)

train_losses, train_accuracies, val_losses, val_accuracies = train_model(resnet_binary, train_binary_loader, val_binary_loader, criterion, optimizer, num_epochs=5, log_interval=10)

NameError: name 'nn' is not defined

In [1]:
test_model(resnet_binary, test_binary_loader, criterion)

NameError: name 'test_model' is not defined